In [ ]:
%env USING_RUN True

%run ./greenhouse/greenhouse.ipynb import Greenhouse

In [ ]:
def run_simulation(greenhouse, date_range, resample_period):
    time_period = pd.to_timedelta(resample_period).total_seconds()

    df = get_weather_data(date_from=date_range[0], date_to=date_range[1], resample_period=resample_period)

    for timestamp, row in df.iterrows():
        try:
            results = greenhouse.run(timestamp, row)
        except Exception as e:
            traceback.print_exc()
            break

        for col in results.keys():
            df.loc[timestamp, col] = results[col]

    return df

In [ ]:
airflow_mode_options = ["humidity_control", "light_control", "constant"]
temp_mode_options = ["constant", "dynamic"]
use_sunlight_options = [True, False]
roof_panel_options = ["polycarbonate", "insolight", "solarbrite"]

In [ ]:
airflow_mode = 'constant'

In [ ]:
temp_mode = 'constant'

In [ ]:
use_sunlight = 'False'

In [ ]:
roof_panel_type = 'polycarbonate'

In [ ]:
airflow_l_per_s = 100

In [ ]:
min_temp = 16

In [ ]:
max_temp = 26

In [ ]:
max_humidity = 70

In [ ]:
resample_period="60min"

airflow = airflow_mode

if airflow_mode == "constant":
    airflow = f"CONST:{airflow_l_per_s / 1000}"

configs = {
    "custom": Greenhouse(
        resample_period=resample_period, 
        airflow_mode=airflow,
        temp_mode=temp_mode,
        min_temp=min_temp,
        max_temp=max_temp,
        max_humidity=max_humidity,
        block_sunlight=False if use_sunlight == "True" else False,
        roof_panel_type=roof_panel_type
    )
}

dates = {
    "winter": ["2020-01-01", "2020-01-02"],
    "spring": ["2020-04-03", "2020-04-04"],
    "summer": ["2020-07-10", "2020-07-11"],
    "fall": ["2020-10-01", "2020-10-02"],
}

dfs = {}
results = []
for config_type in configs:
    dfs[config_type] = {}
    greenhouse = configs[config_type]

    results_of_date = {}
    for date_type in dates:
        date_range = dates[date_type]

        df = run_simulation(greenhouse, date_range, resample_period)

        total_cost, total_yield, total_harvested_plant_count = get_total_cost(df, greenhouse)

        dfs[config_type][date_type] = df
        results_of_date[f"cost_EUR_{date_type}"] = round(total_cost, 2)
        results_of_date[f"yield_kg_{date_type}"] = round(total_yield, 3)
        results_of_date[f"total_harvested_plant_count_{date_type}"] = total_harvested_plant_count

        print(f"Simulation done for {config_type} - {date_type}")

    results.append({
        "config_type": config_type,
        **results_of_date
    })

results_df = pd.DataFrame(results)

In [ ]:
results_df["total_cost"] = round(results_df[[f"cost_EUR_{date_type}" for date_type in dates]].sum(axis=1), 2)
results_df["total_yield"] = round(results_df[[f"yield_kg_{date_type}" for date_type in dates]].sum(axis=1), 3)
results_df["total_plant_count"] = results_df[[f"total_harvested_plant_count_{date_type}" for date_type in dates]].sum(axis=1)
results_df["cost_per_kg"] = round(results_df["total_cost"] / results_df["total_yield"], 2)
results_df["cost_per_plant"] = round(results_df["total_cost"] / results_df["total_plant_count"], 4)
results_df[["config_type", "total_cost", "total_yield", "cost_per_kg", "total_plant_count", "cost_per_plant"]]

In [ ]:
plot_multiline_dual_y(
    dfs["custom"]["spring"],
    ["temp", "ambient_temp"],
    ["heating_rate_J_per_s"],
    width=width, 
    height=height, 
    title=f"Temperature: {title}", 
    y_labels=["C", "J / s"], 
    legend_label=" " if show_label else None,
    date_format="",
    colors=["blue", "green", "orange"],
    left_axis_target_range=[17,27]
)

In [ ]:
plot_in_grid(dfs, "custom", plot_energy)

In [ ]:
plot_in_grid(dfs, "custom", plot_temperature)

In [ ]:
plot_in_grid(dfs, "custom", plot_humidity)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2f5dc715-67f7-4c8c-98f7-a87b736d3338' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>